<a href="https://colab.research.google.com/github/AdiKaipa/Capstone-Project/blob/main/Generation_of_Code_using_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries

In [1]:
import string
import re
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext import legacy
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = '/content/drive/MyDrive/END/Capstone Project/English_Python_Data_Latest.txt'

#Reading data from the text file

In [5]:
def Read_Text(file_name):
  file = open(file_name, mode = 'rt', encoding= 'utf-8')
  text = file.read()
  file.close()
  return text

In [6]:
data_text = Read_Text(path)

In [7]:
data_text[:1000]

"# write a python program to add two numbers \nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n\n# write a python function to add two user provided numbers and return the sum\ndef add_two_numbers(num1, num2):\n    sum = num1 + num2\n    return sum\n\n\n# write a program to find and print the largest among three numbers\n\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 >= num2) and (num1 >= num3):\n   largest = num1\nelif (num2 >= num1) and (num2 >= num3):\n   largest = num2\nelse:\n   largest = num3\nprint(f'largest:{largest}')\n\n\n# write a program to find and print the smallest among three numbers\nnum1 = 10\nnum2 = 12\nnum3 = 14\nif (num1 <= num2) and (num1 <= num3):\n   smallest = num1\nelif (num2 <= num1) and (num2 <= num3):\n   smallest = num2\nelse:\n   smallest = num3\nprint(f'smallest:{smallest}')\n\n\n# Write a python function to merge two given lists into one\ndef merge_lists(l1, l2):\n    return l1 + l2\n\n\n# Write a program to check whether a number is prim

#PreProcessing of the Special Characters

In [8]:
def Special_char_space(data):
  data = data.lower()
  data = data.replace(':', ' : ')
  data = data.replace('=', ' = ')
  data = data.replace('(', ' ( ')
  data = data.replace(')', ' ) ')
  data = data.replace('\n\n\n', '\n')
  return data 
data_text = Special_char_space(data_text)


In [9]:
def PreProcess(data):
  data_latest = re.sub(r'#[0-9]', '#', data)
  data_latest = re.sub(r'#[0-9.]', '#', data_latest)
  data_latest = re.sub(r'# ', '#', data_latest)
  data_latest = re.sub(r'    ', '\t', data_latest)
  return data_latest


In [10]:
data_latest = PreProcess(data_text)

In [11]:
data_latest[:100]

"#write a python program to add two numbers \nnum1  =  1.5\nnum2  =  6.3\nsum  =  num1 + num2\nprint ( f'"

# DataFrame creation with Title and Target Code as Columns

In [12]:
sent = data_latest.split('#')

In [13]:
len(sent)

4650

In [14]:
sent[:100]

['',
 "write a python program to add two numbers \nnum1  =  1.5\nnum2  =  6.3\nsum  =  num1 + num2\nprint ( f'sum :  {sum}' ) \n",
 'write a python function to add two user provided numbers and return the sum\ndef add_two_numbers ( num1, num2 )  : \n\tsum  =  num1 + num2\n\treturn sum\n',
 "write a program to find and print the largest among three numbers\n\nnum1  =  10\nnum2  =  12\nnum3  =  14\nif  ( num1 > =  num2 )  and  ( num1 > =  num3 )  : \n   largest  =  num1\nelif  ( num2 > =  num1 )  and  ( num2 > =  num3 )  : \n   largest  =  num2\nelse : \n   largest  =  num3\nprint ( f'largest : {largest}' ) \n",
 "write a program to find and print the smallest among three numbers\nnum1  =  10\nnum2  =  12\nnum3  =  14\nif  ( num1 < =  num2 )  and  ( num1 < =  num3 )  : \n   smallest  =  num1\nelif  ( num2 < =  num1 )  and  ( num2 < =  num3 )  : \n   smallest  =  num2\nelse : \n   smallest  =  num3\nprint ( f'smallest : {smallest}' ) \n",
 'write a python function to merge two given lists

In [15]:
sent[0] = sent[0].replace('#', '')
sent[0]

''

In [16]:
Title = []
py_code = []
for s in sent:
  words = s.split('\n')
  Title.append(words[0])
  py_code.append(words[1:])

print(len(Title), len(py_code))

4650 4650


In [17]:
'''
for i in range(len(py_code)):
  py_code[i] = ' \n '.join(py_code[i])
'''

"\nfor i in range(len(py_code)):\n  py_code[i] = ' \n '.join(py_code[i])\n"

In [18]:
py_code[:5]

[[],
 ['num1  =  1.5',
  'num2  =  6.3',
  'sum  =  num1 + num2',
  "print ( f'sum :  {sum}' ) ",
  ''],
 ['def add_two_numbers ( num1, num2 )  : ',
  '\tsum  =  num1 + num2',
  '\treturn sum',
  ''],
 ['',
  'num1  =  10',
  'num2  =  12',
  'num3  =  14',
  'if  ( num1 > =  num2 )  and  ( num1 > =  num3 )  : ',
  '   largest  =  num1',
  'elif  ( num2 > =  num1 )  and  ( num2 > =  num3 )  : ',
  '   largest  =  num2',
  'else : ',
  '   largest  =  num3',
  "print ( f'largest : {largest}' ) ",
  ''],
 ['num1  =  10',
  'num2  =  12',
  'num3  =  14',
  'if  ( num1 < =  num2 )  and  ( num1 < =  num3 )  : ',
  '   smallest  =  num1',
  'elif  ( num2 < =  num1 )  and  ( num2 < =  num3 )  : ',
  '   smallest  =  num2',
  'else : ',
  '   smallest  =  num3',
  "print ( f'smallest : {smallest}' ) ",
  '']]

In [19]:
print(py_code[2])

['def add_two_numbers ( num1, num2 )  : ', '\tsum  =  num1 + num2', '\treturn sum', '']


In [20]:
df = pd.DataFrame({'Title': Title, 'Target_code': py_code})
df.shape

(4650, 2)

In [21]:
df.head()

,Title,Target_code
0,,[]
1,write a python program to add two numbers,"[num1 = 1.5, num2 = 6.3, sum = num1 + nu..."
2,write a python function to add two user provid...,"[def add_two_numbers ( num1, num2 ) : , \tsum..."
3,write a program to find and print the largest ...,"[, num1 = 10, num2 = 12, num3 = 14, if ..."
4,write a program to find and print the smallest...,"[num1 = 10, num2 = 12, num3 = 14, if ( ..."


# Embedding vector creation using word2vec model

In [22]:
en = en = spacy.load('en_core_web_sm')
def Tokenize(sentence):
  sentence = str(sentence).replace('\n', '')
  return [tok.text for tok in en.tokenizer(sentence)]

In [23]:
import gensim
w2v_dim = 256
w2v_min_count = 2
w2v_window = 3
target = []
for sent in df['Target_code'].values:
  sent_token = Tokenize(sent)
  target.append(sent_token)
w2v_model = gensim.models.Word2Vec(target, size = w2v_dim, window = w2v_window, min_count = w2v_min_count)


In [24]:
w2v_model.save('embeddings.txt')

# Preparation of Source and Target

In [25]:
SRC = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

TRG = Field(tokenize = 'spacy', 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True, 
            batch_first = True)

In [26]:
fields = [('Title', SRC),('Target_code',TRG)]

In [27]:
example = [legacy.data.Example.fromlist([df.Title[i],df.Target_code[i]], fields) for i in range(df.shape[0])] 

In [28]:
CodeDataset = legacy.data.Dataset(example, fields)

In [29]:
vars(CodeDataset.examples[0])

{'Target_code': [], 'Title': []}

In [30]:
(train, test, val) = CodeDataset.split(split_ratio=[0.80, 0.10, 0.10], random_state=random.seed(SEED))

In [31]:
SRC.build_vocab(train, min_freq = 2)
TRG.build_vocab(train, min_freq = 2)

In [32]:
print("size of SRC vocab: ", len(SRC.vocab))
print('size of TRG vocab: ', len(TRG.vocab))

size of SRC vocab:  1419
size of TRG vocab:  4505


In [33]:
from tqdm import tqdm_notebook
word2vec_vectors = []
for token, idx in tqdm_notebook(TRG.vocab.stoi.items()):
  if token in w2v_model.wv.vocab.keys():
    word2vec_vectors.append(torch.FloatTensor(w2v_model[token]))
  else:
    word2vec_vectors.append(torch.zeros(w2v_dim))
TRG.vocab.set_vectors(TRG.vocab.stoi, word2vec_vectors, w2v_dim)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  """


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train, val, test),
    sort = False, 
     batch_size = BATCH_SIZE,
     device = device)

# Encoder Architecture

In [36]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [37]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [38]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [39]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

#Decoder Architecture

In [40]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 pre_trained_emb,
                 max_length = 500):
        super().__init__()
        
        self.device = device
        self.pre_trained_emb = pre_trained_emb
        self.tok_embedding = nn.Embedding.from_pretrained(self.pre_trained_emb)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
                
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [41]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [42]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        
        enc_src = self.encoder(src, src_mask)
        
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [43]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
pre_trained_emb = torch.FloatTensor(TRG.vocab.vectors)
enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device,
              pre_trained_emb) 


In [44]:
SRC_PAD_IDX = SRC.vocab.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [45]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,628,313 trainable parameters


In [46]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [47]:
model.apply(initialize_weights);

In [48]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [49]:
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [50]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Title
        trg = batch.Target_code
        
        optimizer.zero_grad()
        
        output, _ = model(src, trg[:,:-1])
                
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [51]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.Title
            trg = batch.Target_code

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [52]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [53]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 3s
	Train Loss: 5.243 | Train PPL: 189.329
	 Val. Loss: 3.074 |  Val. PPL:  21.630
Epoch: 02 | Time: 0m 3s
	Train Loss: 4.286 | Train PPL:  72.708
	 Val. Loss: 2.927 |  Val. PPL:  18.670
Epoch: 03 | Time: 0m 3s
	Train Loss: 3.981 | Train PPL:  53.593
	 Val. Loss: 2.724 |  Val. PPL:  15.245
Epoch: 04 | Time: 0m 3s
	Train Loss: 3.668 | Train PPL:  39.187
	 Val. Loss: 2.512 |  Val. PPL:  12.328
Epoch: 05 | Time: 0m 3s
	Train Loss: 3.356 | Train PPL:  28.680
	 Val. Loss: 2.350 |  Val. PPL:  10.484
Epoch: 06 | Time: 0m 3s
	Train Loss: 3.037 | Train PPL:  20.844
	 Val. Loss: 2.144 |  Val. PPL:   8.532
Epoch: 07 | Time: 0m 3s
	Train Loss: 2.758 | Train PPL:  15.762
	 Val. Loss: 2.002 |  Val. PPL:   7.403
Epoch: 08 | Time: 0m 3s
	Train Loss: 2.456 | Train PPL:  11.662
	 Val. Loss: 1.866 |  Val. PPL:   6.460
Epoch: 09 | Time: 0m 3s
	Train Loss: 2.189 | Train PPL:   8.924
	 Val. Loss: 1.734 |  Val. PPL:   5.665
Epoch: 10 | Time: 0m 3s
	Train Loss: 1.923 | Train PPL:   6.845


In [54]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 1.370 | Test PPL:   3.937 |


In [55]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 500):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [56]:
def display_attention(sentence, translation, attention, n_heads = 8, n_rows = 4, n_cols = 2):
    
    assert n_rows * n_cols == n_heads
    
    fig = plt.figure(figsize=(15,25))
    
    for i in range(n_heads):
        
        ax = fig.add_subplot(n_rows, n_cols, i+1)
        
        _attention = attention.squeeze(0)[i].cpu().detach().numpy()

        cax = ax.matshow(_attention, cmap='bone')

        ax.tick_params(labelsize=12)
        ax.set_xticklabels(['']+['<sos>']+[t.lower() for t in sentence]+['<eos>'], 
                           rotation=45)
        ax.set_yticklabels(['']+translation)

        ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
        ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [57]:
src = 'write a python program to sum two numbers'
src = Tokenize(src)
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')
for word in translation:
  print(word)

predicted trg = ['num1  =  1.5', 'num2  =  6.3', 'sum  =  num1 - num2', "print ( f'sub :  {sum}' ) ", '', '', '<eos>']
num1  =  1.5
num2  =  6.3
sum  =  num1 - num2
print ( f'sub :  {sum}' ) 


<eos>


In [58]:
src = 'write a python program to check whether the given string is palindrome'
src = Tokenize(src)
translation, attention = translate_sentence(src, SRC, TRG, model, device)

print(f'predicted trg = {translation}')
for word in translation:
  print(word)

predicted trg = ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 'else : ', '<unk>', '', '', '<eos>']
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
<unk>
else : 
<unk>


<eos>


In [59]:
example_idx = 0

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'python', 'function', 'to', 'check', 'if', 'a', 'given', 'string', 'is', 'a', 'palindrome']
trg = ['', 'def ispalindrome ( s )  : ', '\treturn s  =  =  s[ :  : -1]', '', '']



predicted trg = ['', 'def ispalindrome ( s )  : ', '\treturn s  =  =  s[ :  : -1]', '', '', '<eos>']

def ispalindrome ( s )  : 
	return s  =  =  s[ :  : -1]


<eos>


In [60]:
example_idx = 4

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'python', 'function', 'for', 'implementation', 'of', 'insertion', 'sort']
trg = ['', 'def insertionsort ( arr )  :  ', '', '\tfor i in range ( 1, len ( arr )  )  :  ', '', '\t\tkey  =  arr[i] ', '', '\t\tj  =  i-1', '\t\twhile j > = 0 and key < arr[j]  :  ', '\t\t\t\tarr[j+1]  =  arr[j] ', '\t\t\t\tj - =  1', '\t\tarr[j+1]  =  key ', 'arr  =  [12, 11, 13, 5, 6] ', 'insertionsort ( arr )  ', 'print  ( f"sorted array is :  {arr}" )  ', '']



predicted trg = ['', 'def bubblesort ( arr )  :  ', '\tn  =  len ( arr )  ', '\tfor i in range ( n-1 )  :  ', '\t\tfor j in range ( 0, n-i-1 )  :  ', '', '\t\t\tif arr[j] > arr[j+1]  :  ', '\t\t\t\tarr[j], arr[j+1]  =  arr[j+1], arr[j] ', '', '<unk>', '', '<unk>', '', '<unk>', '', '<unk>', '', '<unk>', '', '<eos>']

def bubblesort ( arr )  :  
	n  =  len ( arr )  
	for i in range ( n-1 )  :  
		for j in range ( 0, n-i-1 )  :  

			if arr[j] > arr[j+1]  :  
				arr[j], arr[j+1]  =  arr[j+1], arr[j] 

<unk>

<unk>

<unk>

<unk>

<

In [61]:
example_idx = 13

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'python', 'function', 'to', 'generate', 'amortization', 'schedule', 'given', 'initial', 'loan', 'amount', ',', 'interest', 'rate', ',', 'annual', 'payments', 'and', 'tenure', '.']
trg = ['import itertools', 'def loan_schedule ( principal, interest_rate, annual_payment, tenure )  : ', '\tif ( tenure < =  0 )  : ', '\t\tprint ( "invalid tenure",tenure ) ', '\t\traise valueerror', '\tif ( interest_rate > 1 or interest_rate < 0 )  : ', '\t\tprint ( "invalid interest rate",interest_rate," expected between 0 and 1" ) ', '\t\traise valueerror', '\tcashflows  =  [principal, *list ( itertools.repeat ( -annual_payment, tenure )  ) ]', '\teffective_interest_rate  =  1+interest_rate', '\treturn [ val for val in list ( itertools.accumulate ( cashflows, lambda bal, pmt :   ( bal*effective_interest_rate + pmt )  )  )  if val > 0]', '', '']



predicted trg = ['import itertools', 'def loan_schedule ( principal, interest_rate, annual_payment, tenure )  : ', '\tif ( tenure < =  0 ) 

In [62]:
example_idx = 18

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'function', 'that', 'returns', 'derivative', 'of', 'sine', 'value', 'of', 'the', 'input']
trg = ['def derivative_sin ( x : float ) -> float : ', '\timport math', '\treturn math.cos ( x ) ', '', '']



predicted trg = ['<unk>', '\timport math', '\treturn math.cos ( x ) ', '', '', '<eos>']
<unk>
	import math
	return math.cos ( x ) 


<eos>


In [63]:
example_idx = 25

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['first', 'two', 'terms']
trg = ['n1, n2  =  0, 1', 'count  =  0', '', '']



predicted trg = ['<unk>', '', '<eos>']
<unk>

<eos>


In [64]:
example_idx = 19

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'python', 'program', 'that', 'asks', 'user', 'to', 'guess', 'a', 'number', 'between', '1', 'and', '5', 'and', 'guess', 'it', 'within', '3', 'guesses']
trg = ['print ( "please guess a number between 1 and 5 and i will guess within 3 chances!" ) ', 'guess1  =  input ( "is it < =  3? enter y/n \\n" ) ', 'if guess1  =  =  "y" : ', '\tguess2  =  input ( "is it < =  2? enter y/n \\n" ) ', '\tif guess2  =  =  "y" : ', '\t\tguess3  =  input ( "is it 1? enter y/n \\n" ) ', '\t\tif guess3  =  =  "y" : ', '\t\t\tprint ( "yay! found the number, its 1" ) ', '\t\telse : ', '\t\t\tprint ( "yay! found the number, its 2" ) ', '\telse : ', '\t\tprint ( "yay! found the number, its 3" ) ', 'else : ', '\tguess2  =  input ( "is it 4? enter y/n \\n" ) ', '\tif guess2  =  =  "y" : ', '\t\tprint ( "yay! found the number, its 4" ) ', '\telse : ', '\t\tprint ( "yay! found the number, its 5" ) ', '']



predicted trg = ['<unk>', '<unk>', '<unk>', '\telse : ', '<unk>', '\tif guess2  =  =  "y" 

In [65]:
example_idx = 31

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['python', 'program', 'to', 'find', 'largest', 'and', 'smallest', 'number', 'in', 'a', 'list']
trg = ['', 'numlist  =  []', 'number  =  int ( input ( "please enter the total number of list elements :  " )  ) ', 'for i in range ( 1, number + 1 )  : ', '\tvalue  =  int ( input ( "please enter the value of %d element  :  " %i )  ) ', '\tnumlist.append ( value ) ', '', 'smallest  =  largest  =  numlist[0]', '', 'for j in range ( 1, number )  : ', '\tif ( smallest > numlist[j] )  : ', '\t\tsmallest  =  numlist[j]', '\t\tmin_position  =  j', '\tif ( largest < numlist[j] )  : ', '\t\tlargest  =  numlist[j]', '\t\tmax_position  =  j', '', 'print ( "the smallest element in this list is  :  ", smallest ) ', 'print ( "the index position of smallest element in this list is  :  ", min_position ) ', 'print ( "the largest element in this list is  :  ", largest ) ', 'print ( "the index position of largest element in this list is  :  ", max_position ) ', '', '']



predicted trg = ['<unk>', '<unk

In [66]:
example_idx = 6

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'program', 'that', 'prints', 'the', 'number', 'of', 'unique', 'keys', 'in', 'a', 'list', 'of', 'dictionaries', '.']
trg = ['list_of_dicts  =  [{"key1" :  "val1", "country" :  "india"}, ', '\t\t\t\t {"country" :  "usa", "foo" :  "bar"},', '\t\t\t\t {"foo" :  "bar", "foo2" : "bar2"}]', 'unique_keys  =  []', 'for d in list_of_dicts : ', '  for key in d : ', '\tif key not in unique_keys : ', '\t  unique_keys.append ( key ) ', 'print ( f"number of unique keys :  {len ( unique_keys ) }" ) ', '', '']



predicted trg = ['list_of_dicts  =  [{"key1" :  "val1", "country" :  "india"}, ', '\t\t\t\t {"country" :  "usa", "foo" :  "bar"},', '\t\t\t\t {"foo" :  "bar", "foo2" : "bar2"}]', 'unique_keys  =  []', 'for d in list_of_dicts : ', '  for key in d : ', '\tif key not in unique_keys : ', '\t  unique_keys.append ( key ) ', 'print ( f"number of unique keys :  {len ( unique_keys ) }" ) ', '', '', '<eos>']
list_of_dicts  =  [{"key1" :  "val1", "country" :  "india"}, 
				 {"countr

In [67]:
example_idx = 22

src = vars(val.examples[example_idx])['Title']
trg = vars(val.examples[example_idx])['Target_code']

print(f'src = {src}')
print(f'trg = {trg}')

translation, attention = translate_sentence(src, SRC, TRG, model, device)
print('\n\n')
print(f'predicted trg = {translation}')
for word in translation:
  print(word)

src = ['write', 'a', 'python', 'program', ' ', 'to', 'convert', 'list', 'of', 'tuples', 'into', 'list']
trg = ['', "lt  =  [ ( 'english', 2 ) ,  ( 'maths', 4 ) ,  ( 'science', '6' ) ] ", 'out  =  [item for t in lt for item in t]  ', 'print ( out ) ', '', '']



predicted trg = ['<unk>', '<unk>', '<unk>', '', '<eos>']
<unk>
<unk>
<unk>

<eos>
